In [ ]:
SELECT current_database() AS DATABASE_NAME, current_schema() AS SCHEMA_NAME


In [ ]:
# Import python packages
import logging
from snowflake.snowpark.context import get_active_session

logger = logging.getLogger("fred_logger")

current_context_df = cells.sql_get_context.to_pandas()
database_name = current_context_df.iloc[0,0]
schema_name = current_context_df.iloc[0,1]

session = get_active_session()

logger.info("02_raw_to_harmonized start")

In [ ]:
import snowflake.snowpark.functions as F

env = schema_name[:3]
def create_fred_view(session):
    raw_fred_10Y = session.table(f"{database_name}.{env}_RAW_FRED.FRED_DGS10Y").select(
                                                                                        F.to_date(F.col('"observation_date"')).alias("OBSERVATION_DATE"),
                                                                                        F.col('"DGS10"').cast("float").alias("10Y_YIELD")
                                                                                    )
    
    raw_fred_2Y = session.table(f"{database_name}.{env}_RAW_FRED.FRED_DGS2Y").select(
                                                                                        F.to_date(F.col('"observation_date"')).alias("OBSERVATION_DATE"),
                                                                                        F.col('"DGS2"').cast("float").alias("2Y_YIELD")
                                                                                    )

    harmonized_data = raw_fred_10Y.join(raw_fred_2Y, raw_fred_10Y['OBSERVATION_DATE'] == raw_fred_2Y['OBSERVATION_DATE']).select(
                                                                                                                                    raw_fred_10Y["OBSERVATION_DATE"].alias("OBSERVATION_DATE"),  # Keep only one OBSERVATION_DATE
                                                                                                                                    F.col("10Y_YIELD"),
                                                                                                                                    F.col("2Y_YIELD")
                                                                                                                                )

    # harmonized_data = harmonized_data.filter(
    #     (F.col("YIELD_SPREAD").isNotNull()))
    
    session.use_schema(f"{database_name}.{schema_name}")
    harmonized_data.write.mode("overwrite").save_as_table('FRED_FLATTENED')

def create_fred_view_stream(session):
    _ = session.sql('CREATE OR REPLACE STREAM FRED_STREAM \
                        ON VIEW FRED_FLATTENED \
                        SHOW_INITIAL_ROWS = TRUE').collect()
    


In [ ]:

create_fred_view(session)
create_fred_view_stream(session)